## Read in data

path:

/xdisk/chanc/share/

there are two folders: GRMHD_dt1000M and GRMHD_dt10M

under GRMHD_dt1000M: (5000-5900, step=100 for reading in files) <br>Ma+0.5_w5 <br>Ma+0.94_w5<br> Ma-0.5_w5<br> Ma-0.94_w5<br> Ma0_w5<br> Sa+0.5_w5<br> Sa+0.94_w5<br> Sa-0.5_w5<br> Sa-0.94_w5<br> Sa0_w5

under GRMHD_dt10M: (5000 - 5999, step=1) <br>Ma+0.94_w5<br>Ma-0.94_w5 <br> Sa+0.94_w5

USE GRMHD_dt1000M first <br>
USE GRMHD_dt10M, step=10 for the calculation of variance and MI

In [2]:
from toolz import valmap

import h5py
import pyharm

from matplotlib   import pyplot as plt
import numpy as np
from pyharm.plots import plot_dumps as pplt

In [3]:
i0 = 5000 # initial  frame
i1 = 5900 # 1 + last frame

fmt = 'torus.out0.{:05d}.h5'

qualities = ['RHO', 'Pg', 'bsq', 'beta', 'Theta', 'Thetae']

In [4]:
def fname(i, src): # closure on `src` and `fmt`
    return (src+fmt).format(i)

## Data (for 1000M)

In [5]:
path_1000M = '/xdisk/chanc/share/GRMHD_dt1000M/'
src_ma = ['Ma+0.5_w5', 'Ma+0.94_w5', 'Ma-0.5_w5', 'Ma-0.94_w5', 'Ma0_w5']
src_sa = ['Sa+0.5_w5', 'Sa+0.94_w5', 'Sa-0.5_w5', 'Sa-0.94_w5', 'Sa0_w5']
src_ma_1000M = []
src_sa_1000M = []
for i in range(len(src_ma)):
    src_ma_1000M.append(path_1000M+src_ma[i]+'/')
    src_sa_1000M.append(path_1000M+src_sa[i]+'/')

dump0 = pyharm.load_dump(fname(i0, src_ma_1000M[0]))
R_low = 1.
R_high = [1, 10, 40, 160]
# first row of SANE model params

In [6]:
def get_dumps(i0, i1, src, step=1):
    dumps = {}
    for i in range(i0, i1, step):
        dumps[str(i)] = pyharm.load_dump(fname(i, src))
    return dumps

In [7]:
# some constants
cl = 29979245800.0 # speed of light in cgs
mp = 1.67262171e-24
kb = 1.3806505e-16

In [8]:
# R
def R(dump, Rh, Rl):
    beta = dump['beta']
    return Rl/(1+beta**2)+Rh*beta**2/(1+beta**2)

In [9]:
# electron temperature
def Te(dump, Rh, Rl, gamma):
    if gamma == 4/3:
        y = 1
        z = 3
        gamma_i = 4/3
        gamma_e = 7/3
    if gamma == 13/9:
        y = 1
        z = 2
        gamma_i = 13/9
        gamma_e = 133/36
    if gamma == 5/3:
        y = 1
        z = 1
        gamma_i = 5/3
        gamma_e = 5/3
    beta = dump['beta']
    u = dump['Pg']/(gamma-1)
    r = R(dump, Rh, Rl)
    numerator = mp*cl**2*u*(gamma_e-1)*(gamma_i-1)
    denom = kb*dump['RHO']*(y*(gamma_i-1)+r*z*(gamma_e-1))
    Te = numerator/denom
    dump.cache['Te'] = Te
    return Te

In [10]:
def average(irange, keys, dumps):
    s = {'Te': np.zeros_like(dump0['Pg'])}
    for i in irange:
        print('i:', i)
        # Because data I/O is expensive, we load the data once ...
        d = dumps[str(i)].cache
        s['Te'] += d['Te'] 
    return valmap(lambda v: v/len(irange), s)

In [11]:
# works; returns an array of arrays with numerical values
# SKIP THIS FUNCTION
def variance(irange, dumps, interval=100):
    ss = {'varTe': np.zeros_like(dump0['Pg'])}
    for i in irange:
        print(i)
        d = dumps[str(i)].cache
        ss['varTe'] += (d['Te']-d['avgTe'])**2
    return valmap(lambda v: v/(len(irange)*interval), ss)

In [12]:
def MI(dump):
    dump.cache['MI'] = np.sqrt(dump.cache['varTe'])/dump.cache['avgTe']

In [13]:
def change_Rh(dumps, irange, idx, interval=100, gamma=4/3):
    '''
    dumps: a dictionary of the dump files in one folder
    irange: keys' range, e.g. range(i0, i1, step)
    idx: index for which Rh to choose from the Rh list
    interval: 5000, 5100, 5200..., so interval=100
    '''
    Te_rh = {} 
    dict_keys = dumps.keys()
    for key in dict_keys:
        temp = Te(dumps[key], R_high[idx], R_low, gamma)
        Te_rh[key] = temp
    Te_avg = average(irange, ['Te'], dumps)
    for dump in dumps.values():
        dump.cache['avgTe'] = np.squeeze(np.array(list(Te_avg.values())))
    for dump in dumps.values():
        Te_var = (dump.cache['Te'] - dump.cache['avgTe'])**2/interval
        dump.cache['varTe'] = np.squeeze(np.array(list(Te_var.values())))
        dump.cache['MI'] = np.squeeze(np.array(list(MI(dump))))
    print('end one folder')
    return dumps

Use dictionary

look for γ: https://arxiv.org/pdf/2202.11721.pdf <br> Assume ideal gas -> $Γ = \frac{5}{3} + \frac{4}{3}(1 - x_{e})$ where $x_{e}$ is the electron fraction. <br> $\gamma = \frac{4}{3}$, $x_{e} = \frac{1}{4}$, so ions:electrons = 3:1 (y=1, z=3; $\gamma_{i}=\gamma=\frac{4}{3}, \gamma_e = \gamma + x_{e} / (1 - x_{e})(5/3 - \gamma) = \frac{7}{3}$) <br> $\gamma = \frac{13}{9}$, $x_{e} = \frac{1}{3}$, so ions:electrons = 2:1 (y=1, z=2; $\gamma_{i}=\gamma=\frac{13}{9}, \gamma_e = \gamma + x_{e} / (1 - x_{e})(5/3 - \gamma) = \frac{133}{36}$) <br> $\gamma = \gamma_{i}=\frac{5}{3}$, so the gas is fully composed of ions (i.e. $\gamma_{e} = 0$) (? y=z=1; $\gamma_{i}=\gamma_{e} = \gamma = \frac{5}{3}$) <br>

In [15]:
i0 = 5000
i1 = 6000
gammas = [4/3, 13/9, 5/3]

# read in dump files; 10 dumps for each
dumps_ma05 = get_dumps(i0, i1, src_ma_1000M[0], 100) 
dumps_ma094 = get_dumps(i0, i1, src_ma_1000M[1], 100)
dumps_ma_05 = get_dumps(i0, i1, src_ma_1000M[2], 100)
dumps_ma_094 = get_dumps(i0, i1, src_ma_1000M[3], 100)
dumps_ma0 = get_dumps(i0, i1, src_ma_1000M[4], 100)

dumps_sa05 = get_dumps(i0, i1, src_sa_1000M[0], 100)
dumps_sa094 = get_dumps(i0, i1, src_sa_1000M[1], 100)
dumps_sa_05 = get_dumps(i0, i1, src_sa_1000M[2], 100)
dumps_sa_094 = get_dumps(i0, i1, src_sa_1000M[3], 100)
dumps_sa0 = get_dumps(i0, i1, src_sa_1000M[4], 100)

In [16]:
print(dumps_ma05.keys())

dict_keys(['5000', '5100', '5200', '5300', '5400', '5500', '5600', '5700', '5800', '5900'])


get the variance and hence MI for each dump obj in each folder

In [ ]:
# fisrt num: which Rh it uses; second num: which gamma it uses. 11: Rh=R_high[0] and gamma=gammas[0]
dumps = {}
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_ma05_{01}".format(ij)] = change_Rh(dumps_ma05, range(i0, i1, 100), i, interval=100, gamma=gammas[j])
        
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_ma094_{01}".format(ij)] = change_Rh(dumps_ma094, range(i0, i1, 100), i, interval=100, gamma=gammas[j])
        
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_ma_05_{01}".format(ij)] = change_Rh(dumps_ma_05, range(i0, i1, 100), i, interval=100, gamma=gammas[j])
        
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_ma_094_{01}".format(ij)] = change_Rh(dumps_ma_094, range(i0, i1, 100), i, interval=100, gamma=gammas[j])
        
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_ma0_{01}".format(ij)] = change_Rh(dumps_ma0, range(i0, i1, 100), i, interval=100, gamma=gammas[j])

In [ ]:
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_sa05_{01}".forsat(ij)] = change_Rh(dumps_sa05, range(i0, i1, 100), i, interval=100, gamsa=gamsas[j])
        
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_sa094_{01}".forsat(ij)] = change_Rh(dumps_sa094, range(i0, i1, 100), i, interval=100, gamsa=gamsas[j])
        
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_sa_05_{01}".forsat(ij)] = change_Rh(dumps_sa_05, range(i0, i1, 100), i, interval=100, gamsa=gamsas[j])
        
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_sa_094_{01}".forsat(ij)] = change_Rh(dumps_sa_094, range(i0, i1, 100), i, interval=100, gamsa=gamsas[j])
        
for i in range(len(R_high)):
    for j in range(len(gammas)):
        dumps["dumps_sa0_{01}".forsat(ij)] = change_Rh(dumps_sa0, range(i0, i1, 100), i, interval=100, gamsa=gamsas[j])

In [19]:
suffix = {}
for i in range(len(R_high)):
    for j in range(len(gammas)):
        suffix[str(i)+str(j)] = '(R_high, γ) = (' + str(R_high[i]) + ', ' + str(gammas[j]) + ')'
        
suffix

{'00': '(R_high, γ) = (1, 1.3333333333333333)',
 '01': '(R_high, γ) = (1, 1.4444444444444444)',
 '02': '(R_high, γ) = (1, 1.6666666666666667)',
 '10': '(R_high, γ) = (10, 1.3333333333333333)',
 '11': '(R_high, γ) = (10, 1.4444444444444444)',
 '12': '(R_high, γ) = (10, 1.6666666666666667)',
 '20': '(R_high, γ) = (40, 1.3333333333333333)',
 '21': '(R_high, γ) = (40, 1.4444444444444444)',
 '22': '(R_high, γ) = (40, 1.6666666666666667)',
 '30': '(R_high, γ) = (160, 1.3333333333333333)',
 '31': '(R_high, γ) = (160, 1.4444444444444444)',
 '32': '(R_high, γ) = (160, 1.6666666666666667)'}

Then dumps.values().cache['MI'] will be the data arrays

## Model (For 1000M)

data size = 10 (# of folders) × 3 (# of γ) × 4 (# of Rh) × 10 (# of files/folder) = 1200

In [ ]:
import tensorflow
from tensorflow import keras
from keras import layers

## Data (for 10M)

## Model (For 10M)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import layers